<div style="color: white;">

## Model Fitting: Likelihood and Chi-Squared

Model fitting is the process of finding the best parameters for a model that describe observed data. Two commonly used statistical methods for this purpose are the **likelihood function** and the **chi-squared ($\chi^2$) statistic**. These methods quantify the agreement between the model and the data, guiding the optimization of model parameters.

---

### Key Concepts

- **Chi-Squared Statistic ($\chi^2$)**:
  - Measures the goodness-of-fit by summing the squared residuals normalized by their uncertainties:
    $$\chi^2 = \sum_i \frac{\left(y_{\text{obs},i} - y_{\text{model},i}\right)^2}{\sigma_i^2}$$
  - A smaller $\chi^2$ indicates a better fit.
  - **Reduced Chi-Squared ($\chi^2_{\text{red}}$)** accounts for degrees of freedom:
    $$\chi^2_{\text{red}} = \frac{\chi^2}{N - p}$$
    where $N$ is the number of data points and $p$ is the number of model parameters.

- **Likelihood Function ($\mathcal{L}$)**:
  - Represents the probability of observing the data given a model:
    $$\mathcal{L}(\theta) = \prod_i \frac{1}{\sqrt{2\pi}\sigma_i} \exp \left( -\frac{\left(y_{\text{obs},i} - y_{\text{model},i}(\theta)\right)^2}{2\sigma_i^2} \right)$$
  - In practice, the **log-likelihood** is maximized to simplify computations:
    $$\ln \mathcal{L}(\theta) = -\frac{1}{2} \sum_i \left[ \ln(2\pi\sigma_i^2) + \frac{\left(y_{\text{obs},i} - y_{\text{model},i}(\theta)\right)^2}{\sigma_i^2} \right]$$

---

### SED Fitting with SEDFitter

The equations above directly relate to fitting the spectral energy distributions (SEDs) of young stellar objects (YSOs):

1. **Observed Data ($y_{\text{obs},i}$)**:
   - Represents the photometric fluxes of YSOs measured in various filters (e.g., Gaia, 2MASS, WISE).
   - These fluxes are the input data for the fitting process.

2. **Model Predictions ($y_{\text{model},i}$)**:
   - Represent the theoretical fluxes predicted by the SED models for specific physical parameters, such as stellar temperature, disk mass, and extinction.
   - The models are pre-computed and convolved with the same filter response functions as the observations.

3. **Uncertainties ($\sigma_i$)**:
   - Correspond to the photometric errors associated with the observed fluxes.
   - These uncertainties weigh the residuals in both $\chi^2$ and likelihood calculations, emphasizing data points with smaller errors.

In this notebook, we will use **SEDFitter** to implement this framework. By minimizing the $\chi^2$ statistic or maximizing the likelihood function, we aim to identify the best-fitting models for the observed SEDs of YSOs. This allows us to infer their physical properties, such as the size and temperature of their central stars, the structure and mass of their circumstellar disks, and the level of extinction in their surrounding environment.

</div>

<div style="color: #FF4500;">

## Installing SEDFitter
Instructions to download SEDFitter are available here: [SEDFitter Installation](https://sedfitter.readthedocs.io/en/stable/installation.html)

## Downloading the SED Models
SEDFitter only includes the code to perform the model fitting, not the models themselves. We will be using the SED models from this paper: [SED Models Paper](https://arxiv.org/abs/1703.05765)

- Navigate to the following data repository: [Zenodo Repository](https://zenodo.org/records/166732)
- Scroll and click on the file 'sp--s-i.tar.gz' (this will download to your computer).
- Go to the location on your computer where the file downloaded.
- Double click on the file so that it unzips/untars.
- You should now have a folder titled 'sp--s-i' that contains `models.conf`, `parameters.fits`, `stellar.fits`, `flux.fits`, and a sub-directory titled 'convolved'.

</div>

In [ ]:
# After installing SEDFitter, you will also need to PIP install the following packages:
# run: pip install pandas 

import numpy as np
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord
from astropy.io import fits
import astropy.units as u
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

## Read in the data file: 'orion_stars.csv'
Data comes from Großschedl et al. 2019: https://www.aanda.org/articles/aa/abs/2019/02/aa32577-18/aa32577-18.html

In [ ]:
path_to_data = './orionA_stars.csv' # Change this to the path of the data file you have downloaded
df = pd.read_csv(path_to_data)

# Preparing the Data
Below we must prepare our data in a specific format specified by SEDFitter, specified here: https://sedfitter.readthedocs.io/en/stable/data.html

This requires us to convert our magnitudes to fluxes, in units of milli-Janskys (mJy): https://en.wikipedia.org/wiki/Jansky

In [ ]:
other_cols = ['name', 'ra', 'dec']
filters_mags = [
    'Jmag', 'Hmag', 'Ksmag',
    'W1mag', 'W2mag', 'W3mag', 'W4mag'
]
filter_names = [
    'J', 'H', 'Ks',
    'W1', 'W2', 'W3', 'W4'
]
# Effective wavelengths (in microns) for each filter
wavelengths = np.array([
    1.235,  # 2MASS J band
    1.662,  # 2MASS H band
    2.159,  # 2MASS Ks band
    3.4,    # WISE W1
    4.6,    # WISE W2
    12.0,   # WISE W3
    22.0    # WISE W4
])

In [ ]:
# Function to compute the flux density from the magnitude

# Zero-point flux densities in Jy for each filter
zero_point_fluxes = np.array([
    1594,  # 2MASS J band (Jmag)
    1024,  # 2MASS H band (Hmag)
    666.7, # 2MASS Ks band (Ksmag)
    309.54, # WISE W1 band
    171.79, # WISE W2 band
    31.674, # WISE W3 band
    8.363  # WISE W4 band
])

# Function to convert magnitude to flux
def mag_to_flux(mag, zero_point):
    if np.isnan(mag):  # Handle NaNs gracefully
        return np.nan
    return (zero_point * 10**(-0.4 * mag))*1000

# Convert magnitudes to flux densities and compute errors
for filter_name, filter_mag, zero_point in zip(filter_names, filters_mags, zero_point_fluxes):
    flux_col_name = f"{filter_name}_flux"
    flux_err_col_name = f"{filter_name}_flux_err"
    
    # Compute flux
    df[flux_col_name] = df[filter_mag].apply(lambda mag: mag_to_flux(mag, zero_point))
    
    # Compute flux error (10% of flux)
    df[flux_err_col_name] = df[flux_col_name] * 0.10

In [ ]:
for fn in filter_names:
    df['{}_flag'.format(fn)] = df['{}mag'.format(fn)].apply(lambda x: int(1) if pd.notnull(x) and x != 0 else int(0))

In [ ]:
fluxes = df[[f"{filter_name}_flux" for filter_name in filter_names]].values
flux_errors = df[[f"{filter_name}_flux_err" for filter_name in filter_names]].values

In [ ]:
wavelengths

In [ ]:
# Plot an example SED
star = df.loc[df.name == 'star_10']
valid_fluxes = star[[f"{filter_name}_flux" for filter_name in filter_names]].values[0]
valid_flux_errors = star[[f"{filter_name}_flux_err" for filter_name in filter_names]].values[0]
valid_flags = star[[f"{filter_name}_flag" for filter_name in filter_names]].values[0]

# Filter out fluxes where the flag is 0
valid_wavelengths = wavelengths[valid_flags == 1]
valid_fluxes = valid_fluxes[valid_flags == 1]
valid_flux_errors = valid_flux_errors[valid_flags == 1]

plt.figure()
plt.scatter(valid_wavelengths, valid_fluxes, c='r')
plt.errorbar(valid_wavelengths, valid_fluxes, yerr=valid_flux_errors, fmt='o', c='r')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Wavelength (microns)')
plt.ylabel('Flux Density (mJy)')
plt.xlim(-100, 100)
plt.show()

In [ ]:
# Create a list of columns in the desired order
all_cols = list(df.columns)
cols_format = ['name', 'ra', 'dec'] + list(df.columns[df.columns.str.endswith('_flag')])
for filter_name in filter_names:
    cols_format.append(f"{filter_name}_flux")
    cols_format.append(f"{filter_name}_flux_err")

# Reorder the dataframe
df_fmt = df[cols_format]

In [ ]:
#stars = df_fmt.iloc[0:20] # run on only the first 20 stars, good for testing
stars = df_fmt
stars.to_csv('stars', sep=' ', index=False, header=False) # Save to a new file in the desired format for SEDFitter

In [ ]:
from astropy import units as u
from sedfitter import fit
from sedfitter.extinction import Extinction

# Define path to models
model_dir = '/Users/cam/Downloads/sp--s-i/'
#model_dir = '/Users/cam/Downloads/sp--smi/'
extinction_file = '/Users/cam/Downloads/kmh94.par'

# Read in extinction law)
extinction = Extinction.from_file(extinction_file, columns=[0, 3],
                                  wav_unit=u.micron, chi_unit=u.cm**2 / u.g)

# Run the Fitter

In [ ]:
# Define filters and apertures
filters = ['2J', '2H', '2K', 'WISE1', 'WISE2', 'WISE3', 'WISE4'] 
#apertures = [4., 4., 4., 8., 8., 16., 22.] * u.arcsec
apertures = [3., 3., 3., 3., 3., 3., 3.] * u.arcsec

# Run the fitting
fit('stars', filters, apertures, model_dir,
    'output.fitinfo',
    extinction_law=extinction,
    distance_range=[0.1, .6] * u.kpc,
    av_range=[0., 40.])

In [ ]:
# from sedfitter import plot
# plot(
#     input_fits = 'output.fitinfo', 
#     output_dir = 'plots_seds'
#     #select_format = ('A', 6)
#     )

In [ ]:
from sedfitter import write_parameters, write_parameter_ranges
# Write out all models with a delta chi^2-chi_best^2 per datapoint < 3
# Write out the min/max ranges corresponding to the above file
write_parameter_ranges('output.fitinfo', 'parameter_ranges.txt',
                       select_format=('F', 3.))

In [ ]:
from astropy.io import ascii

# File path
file_path = "parameter_ranges.txt"

# Read the file with Astropy, skipping problematic parts
data = ascii.read(
    file_path,
    format="basic",  # Basic ASCII format
    guess=False,     # Don't guess the format
    header_start=1,  # Line index for the main header (second header)
    data_start=3     # Line index for the data (after dashed lines)
)

# Convert to Pandas DataFrame for further analysis if needed
df_params = data.to_pandas()

In [ ]:
param_names = ['chi2', 'av', 'scale', 'star.radius', 'star.temperature', 'disk.mass', 'disk.rmax', 'disk.beta', 'disk.p', 'disk.h100', 'scattering', 'inclination']
for i, param_name in enumerate(param_names):
    if i == 0:
        df_params.rename(columns={
            'min': f'{param_name}_min',
            'best': f'{param_name}_best',
            'max': f'{param_name}_max'
        }, inplace=True)
    else:
        df_params.rename(columns={
            f'min_{i}': f'{param_name}_min',
            f'best_{i}': f'{param_name}_best',
            f'max_{i}': f'{param_name}_max'
        }, inplace=True)

In [ ]:
df_final = pd.merge(stars, df_params, left_index=True, right_index=True, suffixes=('', '_y'))
df_final = df_final.loc[:, ~df_final.columns.str.endswith('_y')]
df_final.to_csv('final_results.csv', index=False)

In [ ]:
df_final

In [ ]:
for k in df_final.keys():
    print(k)

In [ ]:
plt.figure()
plt.scatter(df_final.ra, df_final.dec, c=df_final['av_best'], s = 5, vmax = 20)
plt.show()

In [ ]:
plt.figure()
plt.scatter(df_final['av_best'], df_final['star.temperature_best'], c='blue', s = 1)
plt.show()